In [15]:
import paho.mqtt.client as mqtt
import logging
from threading import Thread
import json
from appJar import gui
from stmpy import Machine, Driver

# MQTT broker address
MQTT_BROKER = 'mqtt.item.ntnu.no'
MQTT_PORT = 1883

# Topics for communication
MQTT_TOPIC_INPUT = 'ttm4115/team_1/server'
MQTT_TOPIC_OUTPUT = 'ttm4115/team_1/collar'

cat_list = ["Kitty","Mons","Roger","Purr"]

class CollarComponent:

    def on_connect(self, client, userdata, flags, rc):
        # we just log that we are connected
        self._logger.debug('MQTT connected to {}'.format(client))

    def on_message(self, client, userdata, msg):
        self._logger.debug('Incoming message to topic {}'.format(msg.topic))
        try:
            payload = json.loads(msg.payload.decode("utf-8"))
        except Exception as err:
            self._logger.error('Message not able to unwrap')
            return
        
        # extract command
        command=payload.get('command')
        print(command)
        
        if command == "cat_in":
            print(payload.get('name') + "cat is home")
            self.stm.send(cat_in)
        if command == "inside_fence":
            print(payload.get('name') + "cat is inside fence")
            self.stm.send(inside_fence)
        if command == "outside_fence":
            print(payload.get('name') + "cat is outside fence")
            self.stm.send(outside_fence)
        if command == "start_tracking":
            print(payload.get('name') + "cat i tracked")
            self.stm.send(start_tracking)
            
    def publish_command(self, command, MQTT_TOPIC_INPUT):
            payload = json.dumps(command)
            self._logger.info(command)
            self.mqtt_client.publish(MQTT_TOPIC_INPUT, payload=payload, qos=2)      

    def __init__(self):
        # get the logger object for the component
        self._logger = logging.getLogger(__name__)
        print('logging under name {}.'.format(__name__))
        self._logger.info('Starting Component')

        # create a new MQTT client
        self._logger.debug('Connecting to MQTT broker {} at port {}'.format(MQTT_BROKER, MQTT_PORT))
        self.mqtt_client = mqtt.Client()
        # callback methods
        self.mqtt_client.on_connect = self.on_connect
        self.mqtt_client.on_message = self.on_message
        # Connect to the broker
        self.mqtt_client.connect(MQTT_BROKER, MQTT_PORT)
        # subscribe to proper topic(s) of your choice
        self.mqtt_client.subscribe(MQTT_TOPIC_OUTPUT)
        # start the internal loop to process MQTT messages
        self.mqtt_client.loop_start()
        
        self.create_gui()
        
        

    def create_gui(self):
        def update(btn):
            cat= self.app.getRadioButton("Gato")
            lat= self.app.getEntry("Lat")
            long= self.app.getEntry("Long")
            data = '{}: {} {}'.format(cat,lat,long)
            self.send_data(data)
    
        def btn_press(button):
            if button == "Cancel":
                self.app.stop()
            else:
                self.app.showSubWindow("Cats")

        self.app = gui("Collar Gui")
        self.app.addButtons(["Choose Cat", "Cancel"], btn_press)

        self.app.startSubWindow("Cats")
        for i in cat_list:
            self.app.addRadioButton("Gato", '{}'.format(i))
        self.app.addLabelEntry("Lat")
        self.app.addLabelEntry("Long")
        self.app.addButton("Update", update)
        self.app.stopSubWindow()

        self.app.go()

    #Update of GPS(lat,long) location
    def send_data(self, data):
        print('Location of {}'.format(data))
        


t0 = {'source': 'initial',
       'target': 'low_freq'}

ts1 = {'trigger': 't',
       'source': 'low_freq',
       'target': 'low_freq',
       'effect': 'send_data'}

t1 = {'trigger': 'inside_fence',
       'source': 'low_freq',
       'target': 'norm_freq'}

ts2 = {'trigger': 't',
       'source': 'norm_freq',
       'target': 'norm_freq',
       'effect': 'send_data'}

t2 = {'trigger': 'cat_in',
       'source': 'norm_freq',
       'target': 'low_freq'}

t3 = {'trigger': 'start_tracking',
       'source': 'norm_freq',
       'target': 'high_freq'}

t4 = {'trigger': 'outside_fence',
       'source': 'norm_freq',
       'target': 'high_freq'}

ts3 = {'trigger': 't',
       'source': 'high_freq',
       'target': 'high_freq',
       'effect': 'send_data'}

t5 = {'trigger': 'cat_in',
       'source': 'high_freq',
       'target': 'low_freq'}

t6 = {'trigger': 'inside_fence',
       'source': 'high_freq',
       'target': 'norm_freq'}

low_freq = {'name': 'low_freq',
      'entry': 'start_timer("t", 600000)'}

norm_freq = {'name': 'norm_freq',
      'entry': 'start_timer("t", 60000)'}

high_freq = {'name': 'high_freq',
      'entry': 'start_timer("t", 5000)'}


# logging.DEBUG: Most fine-grained logging, printing everything
# logging.INFO:  Only the most important informational log items
# logging.WARN:  Show only warnings and errors.
# logging.ERROR: Show only error messages.

debug_level = logging.DEBUG
logger = logging.getLogger(__name__)
logger.setLevel(debug_level)
ch = logging.StreamHandler()
ch.setLevel(debug_level)
formatter = logging.Formatter('%(asctime)s - %(name)-12s - %(levelname)-8s - %(message)s')
ch.setFormatter(formatter)
logger.addHandler(ch)

c = CollarComponent()

stm_collar = Machine(name='stm_collar', transitions=[ts1, ts2, ts3, t0, t1, t2, t3, t4, t5, t6], obj=c, states=[low_freq, norm_freq, high_freq])
c.stm = stm_collar
driver = Driver()
driver.add_machine(stm_collar)
driver.start()

c.app.go()

2020-04-16 23:22:41,881 - __main__     - INFO     - Starting Component
2020-04-16 23:22:41,881 - __main__     - INFO     - Starting Component
2020-04-16 23:22:41,881 - __main__     - INFO     - Starting Component
2020-04-16 23:22:41,881 - __main__     - INFO     - Starting Component
2020-04-16 23:22:41,881 - __main__     - INFO     - Starting Component
2020-04-16 23:22:41,881 - __main__     - INFO     - Starting Component
2020-04-16 23:22:41,881 - __main__     - INFO     - Starting Component
2020-04-16 23:22:41,881 - __main__     - INFO     - Starting Component
2020-04-16 23:22:41,881 - __main__     - INFO     - Starting Component
2020-04-16 23:22:41,881 - __main__     - INFO     - Starting Component
2020-04-16 23:22:41,881 - __main__     - INFO     - Starting Component
2020-04-16 23:22:41,881 - __main__     - INFO     - Starting Component
2020-04-16 23:22:41,881 - __main__     - INFO     - Starting Component
2020-04-16 23:22:41,881 - __main__     - INFO     - Starting Component
2020-0

logging under name __main__.
Location of Mons: 12 12


TclError: invalid command name ".!menu"